# supervised learning:


In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import joblib

# 1 Load Preprocessed Data
file_path = "gym_recommendation_processed_deduped.csv"
df = pd.read_csv(file_path)


# 2 Separate Features & Target Variable
target_column = "Fitness Type"
X = df.drop(columns=[target_column])  # Features (user attributes)
y = df[target_column]  # Target variable (Fitness Type)

# 3 Split Data into Training (80%) and Testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4 Train Decision Tree Classifier (ID3 Algorithm)
dt_model = DecisionTreeClassifier(criterion="entropy", random_state=42)
dt_model.fit(X_train, y_train)

# 5 Train Support Vector Machine (SVM - Linear Kernel)
svm_model = SVC(kernel="linear", random_state=42)
svm_model.fit(X_train, y_train)

# 6 Evaluate Both Models
## Predictions
y_pred_dt = dt_model.predict(X_test)
y_pred_svm = svm_model.predict(X_test)

## Accuracy Scores
accuracy_dt = accuracy_score(y_test, y_pred_dt)
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print(f"Decision Tree Accuracy: {accuracy_dt:.2f}")
print(f"SVM Accuracy: {accuracy_svm:.2f}")

## Detailed Performance Report
print("\nDecision Tree Classification Report:\n", classification_report(y_test, y_pred_dt))
print("\nSVM Classification Report:\n", classification_report(y_test, y_pred_svm))

# 7 Save the Best Model for Future Predictions
best_model = dt_model if accuracy_dt > accuracy_svm else svm_model
joblib.dump(best_model, "best_fitness_model.pkl")
print("✅ Best model saved for future predictions.")

# 8 Function to Predict Fitness Type from User Input
def predict_fitness(user_input):
    model = joblib.load("best_fitness_model.pkl")  # Load saved model
    input_df = pd.DataFrame([user_input], columns=X.columns)  # Convert input to DataFrame
    prediction = model.predict(input_df)[0]  # Get prediction
    return prediction

# Example Usage (Test with a Sample Input)
sample_user = {
    "Sex": 1, "Level": 2, "Fitness Goal": 0, "Hypertension": 0, "Diabetes": 0, 
    "Age": 0.5, "Height": 0.7, "Weight": 0.6, "BMI": 0.4
}
print("Predicted Fitness Type:", predict_fitness(sample_user))

Decision Tree Accuracy: 1.00
SVM Accuracy: 1.00

Decision Tree Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1213
           1       1.00      1.00      1.00      1196

    accuracy                           1.00      2409
   macro avg       1.00      1.00      1.00      2409
weighted avg       1.00      1.00      1.00      2409


SVM Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1213
           1       1.00      1.00      1.00      1196

    accuracy                           1.00      2409
   macro avg       1.00      1.00      1.00      2409
weighted avg       1.00      1.00      1.00      2409

✅ Best model saved for future predictions.
Predicted Fitness Type: 1
